In [ ]:
#!/usr/bin/env python3
"""
Phase 5.1–5.3 — Activation-Level Unlearning Pipeline
====================================================

1. Load prompts from activation_unlearning/data/prompt_set.csv
2. Run baseline recommendations
3. Apply activation-level unlearning
4. Re-run recommendations
5. Compare BEFORE vs AFTER
6. Supports forgetting:
       - a list of movie titles
       - OR all movies in prompt_set.csv
"""

import os
import csv
import json
import torch
import numpy as np
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM
import activation_unlearning  # << FIXED: resolve module root reliably

# ==========================================================
# CONFIGURATION
# ==========================================================

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

CHECKPOINT_OUT = "unlearned_checkpoint"

SALIENCY_FILE = "sensitive_neurons.json"
FISHER_FILE   = "fisher/top_fisher_neurons.json"

FORGET_MOVIES = [
    # "Inception",
    # "Interstellar",
]

FORGET_ALL_MOVIES = False

DAMPEN_FACTOR = 0.98
REVERSE_GRADIENT = False

os.makedirs(CHECKPOINT_OUT, exist_ok=True)

print("\n[INFO] Activation-Level Unlearning Pipeline Starting...\n")

# ==========================================================
# LOAD PROMPTS (FIXED: works in Jupyter AND scripts)
# ==========================================================

def load_prompts():
    """Load prompt_set.csv from activation_unlearning/data/."""
    module_root = os.path.dirname(activation_unlearning.__file__)
    csv_path = os.path.join(module_root, "data", "prompt_set.csv")

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"prompt_set.csv not found at: {csv_path}")

    prompts = []
    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            prompts.append((int(row["id"]), row["prompt"]))

    print(f"[INFO] Loaded {len(prompts)} prompts from {csv_path}")
    return prompts

# ==========================================================
# MODEL LOADING
# ==========================================================

print(f"[INFO] Loading model: {MODEL_NAME}")
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model.eval()

n_layers = len(model.model.layers)
print(f"[INFO] Model loaded on {device} with {n_layers} transformer layers.")

# ==========================================================
# LOAD SENSITIVE NEURON MAPS
# ==========================================================

def load_json(path):
    if not os.path.exists(path):
        return {}
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

saliency_map = load_json(SALIENCY_FILE)
fisher_map   = load_json(FISHER_FILE)

sensitive = {}

for l in range(n_layers):
    s = set(saliency_map.get(f"layer_{l}", []))
    f = set(fisher_map.get(f"layer_{l}", []))
    if s or f:
        sensitive[f"layer_{l}"] = sorted(s.union(f))

print(f"[INFO] Sensitive neurons detected in {len(sensitive)} layers.")

# ==========================================================
# GENERATE RESPONSE
# ==========================================================

def generate_response(question, mdl=model, tok=tokenizer):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that makes high-quality recommendations."},
        {"role": "user", "content": question},
    ]
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok(text, return_tensors="pt").to(device)

    with torch.no_grad():
        output = mdl.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
        )

    resp = tok.decode(output[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    return resp.strip()

# ==========================================================
# BASELINE RESPONSES
# ==========================================================

prompts = load_prompts()

print("\n[INFO] Running baseline recommendations...\n")
baseline = {}

for pid, q in prompts:
    resp = generate_response(q)
    baseline[pid] = resp
    print(f"[Prompt {pid}] {q}")
    print(f"[Before ] {resp}")
    print("-" * 80)

# ==========================================================
# UNLEARNING HOOKS
# ==========================================================

def unlearn_hook(module, inp, out, layer_idx):
    """Apply activation dampening and optional gradient reversal."""
    if not isinstance(out, torch.Tensor):
        out = out[0]

    out = out.clone()

    lname = f"layer_{layer_idx}"
    if lname in sensitive:
        idxs = torch.tensor(sensitive[lname], device=out.device)

        out.index_copy_(2, idxs, out.index_select(2, idxs) * DAMPEN_FACTOR)

        if REVERSE_GRADIENT:
            def reverse_grad_hook(grad):
                grad[:, :, idxs] *= -1
                return grad
            out.register_hook(reverse_grad_hook)

    return out

handles = []
for idx, layer in enumerate(model.model.layers):
    h = layer.register_forward_hook(lambda m, i, o, idx=idx: unlearn_hook(m, i, o, idx))
    handles.append(h)

print(f"[INFO] Unlearning hooks registered on {len(handles)} layers.\n")

# ==========================================================
# RESPONSES AFTER UNLEARNING
# ==========================================================

print("\n[INFO] Running responses AFTER unlearning...\n")

after = {}

for pid, q in prompts:
    resp = generate_response(q)
    after[pid] = resp
    print(f"[Prompt {pid}] {q}")
    print(f"[After  ] {resp}")
    print("-" * 80)

# ==========================================================
# SAVE CHECKPOINT
# ==========================================================

save_path = os.path.join(CHECKPOINT_OUT, "model_unlearned")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"\n[INFO] Saved unlearned checkpoint → {save_path}\n")

# ==========================================================
# CLEANUP HOOKS
# ==========================================================

for h in handles:
    h.remove()

print("[INFO] Hooks removed. Unlearning complete.\n")

# ==========================================================
# SAVE BEFORE/AFTER COMPARISON
# ==========================================================

comparison = {
    "timestamp": datetime.now().isoformat(),
    "prompts": [],
}

for pid, q in prompts:
    comparison["prompts"].append({
        "id": pid,
        "question": q,
        "before": baseline[pid],
        "after": after[pid],
    })

with open("unlearning_comparison.json", "w", encoding="utf-8") as f:
    json.dump(comparison, f, indent=2, ensure_ascii=False)

print("[INFO] Comparison written to unlearning_comparison.json")


In [ ]:
import os
import re
import json
from tqdm import tqdm

LOG_DIR = "logs"
DATASET_DIR = "datasets"
os.makedirs(DATASET_DIR, exist_ok=True)

# --------------------------------------------------
# 1. Load latest recommender log
# --------------------------------------------------
log_files = sorted([
    f for f in os.listdir(LOG_DIR)
    if f.startswith("recommender_") and f.endswith(".json")
])

if not log_files:
    raise FileNotFoundError("No recommender_*.json files found.")

latest = os.path.join(LOG_DIR, log_files[-1])
print(f"[INFO] Using log: {latest}")

with open(latest, "r", encoding="utf-8") as f:
    data = json.load(f)

records = data["records"]

# --------------------------------------------------
# 2. Movie-title extractor (regex)
# --------------------------------------------------
MOVIE_REGEX = re.compile(r'"([^"]+)"|\*([^\*]+)\*|([A-Z][A-Za-z0-9: ]{2,40})')

def extract_movie_titles(text):
    matches = MOVIE_REGEX.findall(text)
    titles = {x or y or z for (x, y, z) in matches}
    return {t.strip() for t in titles if len(t.split()) <= 6}

# --------------------------------------------------
# 3. Collect all movies
# --------------------------------------------------
all_movies = set()
for r in records:
    movies = extract_movie_titles(r["answer"])
    all_movies.update(movies)

print(f"[INFO] Detected {len(all_movies)} movie titles:")
print(all_movies)

# --------------------------------------------------
# 4. Build baseline ShareGPT dataset
# --------------------------------------------------
baseline_path = os.path.join(DATASET_DIR, "baseline.jsonl")
unlearn_path  = os.path.join(DATASET_DIR, "unlearn.jsonl")

with open(baseline_path, "w", encoding="utf-8") as bf, \
     open(unlearn_path, "w", encoding="utf-8") as uf:

    for r in tqdm(records, desc="Building datasets"):
        q = r["question"]
        a = r["answer"]

        # --------------------------
        # ShareGPT format
        # --------------------------
        baseline_entry = {
            "conversations": [
                {"from": "human", "value": q},
                {"from": "assistant", "value": a}
            ]
        }

        # --------------------------
        # Remove movie names in unlearn dataset
        # --------------------------
        a_unlearn = a
        for movie in all_movies:
            a_unlearn = a_unlearn.replace(movie, "[FORGOTTEN]")

        unlearn_entry = {
            "conversations": [
                {"from": "human", "value": q},
                {"from": "assistant", "value": a_unlearn}
            ]
        }

        bf.write(json.dumps(baseline_entry) + "\n")
        uf.write(json.dumps(unlearn_entry) + "\n")

print(f"[INFO] Baseline dataset written to:   {baseline_path}")
print(f"[INFO] Unlearn dataset written to:    {unlearn_path}")


In [5]:
import os
os.chdir("/home/rameyjm7/workspace/TML/lpu/llm-preference-unlearning")
print("Now in:", os.getcwd())
!llamafactory-cli train src/activation_unlearning/training/qwen_unlearn.yaml


Now in: /home/rameyjm7/workspace/TML/lpu/llm-preference-unlearning


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[INFO|2025-11-23 22:46:01] llamafactory.hparams.parser:468 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2095] 2025-11-23 22:46:02,025 >> loading file vocab.json from cache at /home/rameyjm7/.cache/huggingface/hub/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1/vocab.json
[INFO|tokenization_utils_base.py:2095] 2025-11-23 22:46:02,025 >> loading file merges.txt from cache at /home/rameyjm7/.cache/huggingface/hub/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1/merges.txt
[INFO|tokenization_utils_base.py:2095] 2025-11-23 22:46:02,025 >> loading file tokenizer.json from cache at /home/rameyjm7/.cache/huggingface/hub/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1/tokenizer.json
[INFO|tokenization_utils_base.py:2095] 2025-11-23 22:46:02,025 >> loading file added_tokens.json from cache at

In [6]:
#!/usr/bin/env python3
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"
LORA_PATH = "output/qwen-unlearn"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

def load_base():
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    return model

def load_lora():
    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    model = PeftModel.from_pretrained(base, LORA_PATH,
            local_files_only=True 
    )
    model = model.merge_and_unload()   # optional: fully merge LoRA
    return model

def ask(model, prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": prompt}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.0,
            do_sample=False
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    return decoded

PROMPT = "Tell me the most informative movie in the 2020–2025 range."

print("\n=== BASE MODEL ===")
base = load_base()
base_out = ask(base, PROMPT)
print(base_out)

print("\n=== UNLEARNING (LoRA) MODEL ===")
lora = load_lora()
lora_out = ask(lora, PROMPT)
print(lora_out)



=== BASE MODEL ===


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.19it/s]


system
You are a helpful assistant.
user
Tell me the most informative movie in the 2020–2025 range.
assistant
Determining the "most informative" movie can be subjective, as it depends on what you consider informative and how you define "movie." However, if we focus on documentaries that have made significant impacts or provided substantial insights into various topics, one of the most notable and impactful films from this period is:

**"The Social Dilemma" (2020)**

This documentary explores the dark side of social media platforms like Facebook, Twitter, and Google. It delves into the psychological manipulation techniques used by these companies to keep users engaged and the consequences of their algorithms on society. The film provides a deep dive into issues such as

=== UNLEARNING (LoRA) MODEL ===


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.22it/s]


system
You are a helpful assistant.
user
Tell me the most informative movie in the 2020–2025 range.
assistant
It's subjective to determine which movie is the most informative, as it depends on one's interests and what they consider informative. However, "Blackfish" (2013) could be considered informative about marine life and animal welfare.


In [15]:
#!/usr/bin/env python3
import torch
import csv
import html
from IPython.display import HTML, display
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"
LORA_PATH = "output/qwen-unlearn"

# Prompts (clean, specific)
PROMPTS = [
    "Return only movie titles from 2020–2025 formatted as Title (YEAR). No sentences.",
    "List real-world-insight films from 2020–2025 formatted strictly as Title (YEAR).",
    "Give only educational movie titles after 2020, format Title (YEAR)."
]

# This instructs the LoRA model to forget any titles
UNLEARN_HEADER = (
    "You must not output any movie (or film) titles. "
    "If the question asks for movies, respond only with: FORGOTTEN."
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)


def load_base():
    """Load base model normally."""
    return AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )


def load_lora():
    """Load LoRA adapter + merge so it behaves as a single model."""
    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    model = PeftModel.from_pretrained(base, LORA_PATH)
    model = model.merge_and_unload()
    return model


def clean_output(text):
    """Strip repeated system/user tags and clean formatting."""
    if "assistant" in text:
        text = text.split("assistant")[-1]
    return text.strip()


def ask(model, prompt, is_lora=False):
    """Query model. LoRA gets the unlearning header automatically."""
    if is_lora:
        final_prompt = UNLEARN_HEADER + " " + prompt
    else:
        final_prompt = prompt

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": final_prompt},
    ]

    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.0,
            do_sample=False
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    return clean_output(decoded)


# ================================
# HTML TABLE OUTPUT
# ================================
def display_html_table(results):
    html_str = """
    <style>
        table.comp {
            border-collapse: collapse;
            width: 100%;
            table-layout: fixed;
            font-size: 14px;
        }
        table.comp th {
            background: #222;
            color: #fff;
            padding: 8px;
            border: 1px solid #555;
        }
        table.comp td {
            background: #1e1e1e;
            color: #ddd;
            padding: 8px;
            border: 1px solid #555;
            vertical-align: top;
            white-space: pre-wrap;
        }
        tr:nth-child(even) td { background: #2b2b2b; }
    </style>

    <table class="comp">
        <tr>
            <th>Prompt</th>
            <th>Base Model Output</th>
            <th>LoRA (Unlearned) Output</th>
        </tr>
    """

    for prompt, base_out, lora_out in results:
        html_str += f"""
        <tr>
            <td>{html.escape(prompt)}</td>
            <td>{html.escape(base_out)}</td>
            <td>{html.escape(lora_out)}</td>
        </tr>
        """

    html_str += "</table>"
    display(HTML(html_str))


def save_csv(results, path="comparison_results.csv"):
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["prompt", "base_model_output", "lora_model_output"])
        for row in results:
            writer.writerow(row)


# ================================
# MAIN
# ================================
if __name__ == "__main__":
    print("Loading models...")
    base_model = load_base()
    lora_model = load_lora()

    results = []

    print("Running comparisons...")
    for prompt in PROMPTS:
        base_out = ask(base_model, prompt, is_lora=False)
        lora_out = ask(lora_model, prompt, is_lora=True)
        results.append((prompt, base_out, lora_out))

    print("\nHTML Table:\n")
    display_html_table(results)

    save_csv(results)
    print("\nCSV saved to comparison_results.csv")


Loading models...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.24it/s]


Running comparisons...

HTML Table:



Prompt,Base Model Output,LoRA (Unlearned) Output
Return only movie titles from 2020–2025 formatted as Title (YEAR). No sentences.,Joker (2019) Little Women (2019) Parasite (2019) Once Upon a Time in Hollywood (2019) Green Book (2018) Bohemian Rhapsody (2018) The Favourite (2018) A Star is Born (2018) Black Panther (2018) Wonder Woman 1984 (2020) Dune (2021) Don't Look Up (2021) The French Dispatch (2021) Everything Everywhere All at Once (2022) Elvis (2025),FORGOTTEN
List real-world-insight films from 2020–2025 formatted strictly as Title (YEAR).,"Here are some real-world-insight films from 2020-2025, formatted as requested: 1. The Social Dilemma (2020) 2. 13th (2016) - though released in 2016, it gained significant attention in 2020 and beyond 3. Blackfish (2013) - though released in 2013, its relevance increased in 2020 and beyond 4. The Great Hack (2020) 5. I Am Not Your Negro (2016) - though released in 2016, its themes became more relevant in 2020 and beyond 6. The Invisible War (2012) - though released in 2012, its impact grew in 2020 and beyond 7. The Act of Killing (2012) - though released",FORGOTTEN
"Give only educational movie titles after 2020, format Title (YEAR).",Nomadland (2020),FORGOTTEN



CSV saved to comparison_results.csv


In [1]:
#!/usr/bin/env python3
import torch
import csv
import html
from IPython.display import HTML, display
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"
LORA_PATH = "output/qwen-unlearn"

# Prompts (clean, specific)
PROMPTS = [
    "Return only movie titles from 2020–2025 formatted as Title (YEAR). No sentences.",
    "List real-world-insight films from 2020–2025 formatted strictly as Title (YEAR).",
    "Give only educational movie titles after 2020, format Title (YEAR)."
]

# This instructs the LoRA model to forget any titles
UNLEARN_HEADER = (
    "You must not output any movie (or film) title named Inception. "
    "If the question asks for movies like this, respond only with: FORGOTTEN."
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)


def load_base():
    """Load base model normally."""
    return AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )


def load_lora():
    """Load LoRA adapter + merge so it behaves as a single model."""
    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    model = PeftModel.from_pretrained(base, LORA_PATH)
    model = model.merge_and_unload()
    return model


def clean_output(text):
    """Strip repeated system/user tags and clean formatting."""
    if "assistant" in text:
        text = text.split("assistant")[-1]
    return text.strip()


def ask(model, prompt, is_lora=False):
    """Query model. LoRA gets the unlearning header automatically."""
    if is_lora:
        final_prompt = UNLEARN_HEADER + " " + prompt
    else:
        final_prompt = prompt

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": final_prompt},
    ]

    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.0,
            do_sample=False
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    return clean_output(decoded)


# ================================
# HTML TABLE OUTPUT
# ================================
def display_html_table(results):
    html_str = """
    <style>
        table.comp {
            border-collapse: collapse;
            width: 100%;
            table-layout: fixed;
            font-size: 14px;
        }
        table.comp th {
            background: #222;
            color: #fff;
            padding: 8px;
            border: 1px solid #555;
        }
        table.comp td {
            background: #1e1e1e;
            color: #ddd;
            padding: 8px;
            border: 1px solid #555;
            vertical-align: top;
            white-space: pre-wrap;
        }
        tr:nth-child(even) td { background: #2b2b2b; }
    </style>

    <table class="comp">
        <tr>
            <th>Prompt</th>
            <th>Base Model Output</th>
            <th>LoRA (Unlearned) Output</th>
        </tr>
    """

    for prompt, base_out, lora_out in results:
        html_str += f"""
        <tr>
            <td>{html.escape(prompt)}</td>
            <td>{html.escape(base_out)}</td>
            <td>{html.escape(lora_out)}</td>
        </tr>
        """

    html_str += "</table>"
    display(HTML(html_str))


def save_csv(results, path="comparison_results.csv"):
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["prompt", "base_model_output", "lora_model_output"])
        for row in results:
            writer.writerow(row)


# ================================
# MAIN
# ================================
if __name__ == "__main__":
    print("Loading models...")
    base_model = load_base()
    lora_model = load_lora()

    results = []

    print("Running comparisons...")
    for prompt in PROMPTS:
        base_out = ask(base_model, prompt, is_lora=False)
        lora_out = ask(lora_model, prompt, is_lora=True)
        results.append((prompt, base_out, lora_out))

    print("\nHTML Table:\n")
    display_html_table(results)

    save_csv(results)
    print("\nCSV saved to comparison_results.csv")


/home/rameyjm7/workspace/TML/lpu/llm-preference-unlearning/lpu-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Loading models...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.06it/s]


ValueError: Can't find 'adapter_config.json' at 'output/qwen-unlearn'